<a href="https://colab.research.google.com/github/KittLao/sakila/blob/main/sakila.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install mysql-connector-python

In [ ]:
import mysql.connector


In [ ]:
# Database different from database identifier. Identifier contains multiple
# databases.
USER = "admin"
PASSWORD = "21223032315684kK"
HOST = "sakila-mysql.cdfvrj0gw8cb.us-east-1.rds.amazonaws.com"
DATABASE = "sakila"

cnx = mysql.connector.connect(user=USER, password=PASSWORD,
                              host=HOST,
                              database=DATABASE)



In [ ]:
# Need to use buffered cursor so we can execute initialize cursor again if needed.
cursor = cnx.cursor(buffered=True)

# Tables

In [ ]:
show_table_query = ("show tables")
cursor.execute(show_table_query)
tables = cursor.fetchall()
print(tables)

# How much data


In [ ]:
# fetchall gets all records from previous executed query.
# Just use f strings.

def record_count_per_table(cursor, table):
  query = f"select count(*) from {table}"
  cursor.execute(query)
  result = cursor.fetchall()
  return result[0][0]

for table in tables:
  table = table[0]
  count = record_count_per_table(cursor, table)
  print(table, count)


# Columns for Table

In [ ]:
def record_count_per_table(cursor, table):
  query = f"show columns from {table}"
  cursor.execute(query)
  result = cursor.fetchall()
  return result

for table in tables:
  table = table[0]
  columns = record_count_per_table(cursor, table)
  print(table, columns)


# Time period for Payments

In [ ]:
time_payment_query = ("select min(payment_date) as first, max(payment_date) as last from payment")
cursor.execute(time_payment_query)
result =  cursor.fetchall()
print(result)

# Where do events occer? How many are from these locations?


*   District
*   Address
*   City
*   Postal
*   Country





In [ ]:
location_query = """
select
  a.address as address,
  a.district as district,
  a.postal_code as postal_code,
  ci.city as city,
  co.country as country
from address as a
inner join city as ci
on a.city_id = ci.city_id
inner join country as co
on ci.country_id = co.country_id
"""
cursor.execute(location_query)
result = cursor.fetchmany(5)
for record in result:
  print(record)

location_analysis_query = f"""
with locations as (
  {location_query}
)
select
	count(address) as address_count,
	count(district) as district_count,
	count(postal_code) as postal_code_count,
	count(city) as city_count,
	count(country) as country_count
from locations
group by address, district, postal_code, city, country
"""

cursor.execute(location_analysis_query)
result = cursor.fetchmany(5)
for record in result:
  print(record)


# Movie for every Payment

In [ ]:
movie_per_payment_query = """
select
	p.payment_id as payment,
	f.title as movie
from payment as p
inner join rental as r on p.rental_id = r.rental_id
inner join inventory as i on r.inventory_id = i.inventory_id
inner join film as f on i.film_id = f.film_id
"""

cursor.execute(movie_per_payment_query)
for record in cursor.fetchmany(5):
  print(record)

# Sum movie revenue

In [ ]:
revenue_per_movie_query = """
select
	f.title as movie,
	sum(p.amount) as total_revenue
from payment as p
inner join rental as r on p.rental_id = r.rental_id
inner join inventory as i on r.inventory_id = i.inventory_id
inner join film as f on i.film_id = f.film_id
group by movie
order by total_revenue desc
"""

cursor.execute(revenue_per_movie_query)
for record in cursor.fetchmany(5):
  print(record)

# Top grossing cities


In [ ]:
revenue_per_city_query = """
select ci.city as city, sum(p.amount) as revenue
from payment as p
inner join customer as cu on p.customer_id = cu.customer_id
inner join address as a on a.address_id = cu.address_id
inner join city as ci on a.city_id = ci.city_id
group by  ci.city_id
order by revenue desc
"""

cursor.execute(revenue_per_city_query)
for record in cursor.fetchmany(5):
  print(record)

# Revenue of movie per city, customer and month

In [ ]:
movie_revenue_per_city_customer_month_query = """
select
	f.title as title,
	ci.city as city_name,
	CONCAT_WS(' ', cu.first_name, cu.last_name) as customer_name,
	DATE_FORMAT(p.payment_date, "%Y-%M") as month_,
	sum(p.amount) as revenue
from payment as p
inner join rental as r on p.rental_id = r.rental_id
inner join inventory as i on r.rental_id = i.inventory_id
inner join film as f on i.film_id = f.film_id
inner join customer as cu on p.customer_id = cu.customer_id
inner join address as a on a.address_id = cu.address_id
inner join city as ci on a.city_id = ci.city_id
group by title, city_name, customer_name, month_
order by month_ asc, revenue desc
"""

cursor.execute(movie_revenue_per_city_customer_month_query)
for record in cursor.fetchmany(5):
  print(record)

# Facts and Dimensions

Fact should be a spend and dimensions should be details about the entities regarding spend

In [ ]:
drop_date_dim_if_exist = "drop table if exists dimDate"
cursor.execute(drop_date_dim_if_exist)
create_date_dimension = """
create table dimDate
(
	date_key integer not null primary key,
	date_ date not null,
	year_ smallint not null,
	quarter_ smallint not null,
	month_ smallint not null,
	day_ smallint not null,
	week_ smallint not null,
	is_weekend boolean
);
"""
cursor.execute(create_date_dimension)


In [ ]:
drop_movie_dim_if_exist = "drop table if exists dimMovie"
cursor.execute(drop_movie_dim_if_exist)
create_movie_dimension = """
create table dimMovie
(
  movie_key          SERIAL PRIMARY KEY,
  film_id            smallint NOT NULL,
  title              varchar(255) NOT NULL,
  description        text,
  release_year       year,
  language           varchar(20) NOT NULL,
  original_language  varchar(20),
  rental_duration    smallint NOT NULL,
  length             smallint NOT NULL,
  rating             varchar(5) NOT NULL,
  special_features   varchar(60) NOT NULL
);
"""
cursor.execute(create_movie_dimension)


In [ ]:
drop_customer_dim_if_exist = "drop table if exists dimCustomer"
cursor.execute(drop_customer_dim_if_exist)
create_customer_dimension = """
create table dimCustomer
(
  customer_key SERIAL PRIMARY KEY,
  customer_id  smallint NOT NULL,
  first_name   varchar(45) NOT NULL,
  last_name    varchar(45) NOT NULL,
  email        varchar(50),
  address      varchar(50) NOT NULL,
  address2     varchar(50),
  district     varchar(20) NOT NULL,
  city         varchar(50) NOT NULL,
  country      varchar(50) NOT NULL,
  postal_code  varchar(10),
  phone        varchar(20) NOT NULL,
  active       smallint NOT NULL,
  create_date  timestamp NOT NULL,
  start_date   date NOT NULL,
  end_date     date NOT NULL
);
"""
cursor.execute(create_customer_dimension)


In [ ]:
drop_store_dim_if_exist = "drop table if exists dimStore"
cursor.execute(drop_store_dim_if_exist)
create_store_dimension = """
create table dimStore
(
  store_key           SERIAL PRIMARY KEY,
  store_id            smallint NOT NULL,
  address             varchar(50) NOT NULL,
  address2            varchar(50),
  district            varchar(20) NOT NULL,
  city                varchar(50) NOT NULL,
  country             varchar(50) NOT NULL,
  postal_code         varchar(10),
  manager_first_name  varchar(45) NOT NULL,
  manager_last_name   varchar(45) NOT NULL,
  start_date          date NOT NULL,
  end_date            date NOT NULL
);
"""
cursor.execute(create_store_dimension)


In [ ]:
drop_sales_fact_if_exist = "drop table if exists factSales"
cursor.execute(drop_sales_fact_if_exist)
create_sales_fact = """
create table factSales
(
  sales_key        SERIAL PRIMARY KEY,
  date_key         INT NOT NULL REFERENCES dimDate(date_key),
  customer_key     INT NOT NULL REFERENCES dimCustomer(customer_key),
  movie_key        INT NOT NULL REFERENCES dimMovie(movie_key),
  store_key        INT NOT NULL REFERENCES dimStore(store_key),
  sales_amount     decimal(5,2) NOT NULL
);
"""
cursor.execute(create_sales_fact)


In [ ]:
cursor.execute("show tables")
cursor.fetchall()

# ETL from 3NF to Star Schema


*   Dates should be from payment.
*   Customer from customer, city, address and country.
*   Movie from film.
*   Store from address, city, country, store and staff. Start and end date are when store first open and when closed down.
*   Fact table is a person making a sale. Customer, payment, rental, inventory.



In [ ]:
date_etl_query = """
INSERT INTO dimDate (date_key, date_, year_, quarter_, month_, day_, week_, is_weekend)
SELECT distinct(unix_timestamp(payment_date))                     AS date_key,
       date(payment_date)                                           AS date,
       EXTRACT(year FROM payment_date)                              AS year,
       EXTRACT(quarter FROM payment_date)                           AS quarter,
       EXTRACT(month FROM payment_date)                             AS month,
       EXTRACT(day FROM payment_date)                               AS day,
       EXTRACT(week FROM payment_date)                              AS week,
       CASE WHEN dayofweek(payment_date) in (1, 7) THEN true ELSE false END AS is_weekend
FROM payment;
"""
cursor.execute(date_etl_query)
cnx.commit()


In [ ]:
cursor.execute("select * from dimDate limit 5")
for record in cursor.fetchall():
  print(record)


In [ ]:
customer_etl_query = """
insert into dimCustomer (
  customer_key, customer_id, first_name, last_name, email,
  address, address2, district, city, country, postal_code,
  phone, active, create_date, start_date, end_date
)
select
  cu.customer_id as customer_key,
  cu.customer_id as customer_id,
  cu.first_name as first_name,
  cu.last_name as last_name,
  cu.email as email,
  a.address as address,
  a.address2 as address2,
  a.district as district,
  ci.city as city,
  co.country as country,
  a.postal_code as postal_code,
  a.phone as phone,
  cu.active as active,
  cu.create_date as create_date,
  now() as start_date,
  now() as end_date
from customer as cu
inner join address as a on cu.address_id = a.address_id
inner join city as ci on a.city_id = ci.city_id
inner join country as co on ci.country_id = co.country_id
"""
cursor.execute(customer_etl_query)
cnx.commit()


In [ ]:
cursor.execute("select * from dimCustomer limit 5")
for record in cursor.fetchall():
  print(record)

In [ ]:
movie_etl_query = """
insert into dimMovie (
  movie_key, film_id, title, description, release_year, language,
  original_language, rental_duration, length, rating, special_features
)
select
  f.film_id as movie_key, f.film_id, f.title, f.description, f.release_year, l.name as language,
  orig_l.name as original_language, f.rental_duration, f.length, f.rating, f.special_features
from film as f
inner join language as l on f.language_id = l.language_id
left join language as orig_l on f.original_language_id = orig_l.language_id
"""
cursor.execute(movie_etl_query)
cnx.commit()


In [ ]:
cursor.execute("select * from dimMovie limit 5")
for record in cursor.fetchall():
  print(record)

In [ ]:
store_etl_query = """
insert into dimStore (
  store_key, store_id, address, address2, district, city,
  country, postal_code, manager_first_name, manager_last_name,
  start_date, end_date
)
select
  sto.store_id as store_key, sto.store_id, a.address, a.address2,
  a.district, ci.city, co.country, a.postal_code, sta.first_name as manager_first_name,
  sta.last_name as manager_last_name, now() as start_date, now() as end_date
from store as sto
inner join address as a on sto.address_id = a.address_id
inner join city as ci on a.city_id = ci.city_id
inner join country as co on ci.country_id = co.country_id
inner join staff as sta on sto.manager_staff_id = sta.staff_id
"""
cursor.execute(store_etl_query)
cnx.commit()


In [ ]:
cursor.execute("select * from dimStore limit 5")
for record in cursor.fetchall():
  print(record)

In [ ]:
sales_etl_query = """
insert into factSales (
  date_key,
  customer_key,
  movie_key,
  store_key,
  sales_amount
)
select
  UNIX_TIMESTAMP(p.payment_date) as date_key,
  p.customer_id as customer_key,
  i.film_id as movie_key,
  i.store_id as store_key,
  p.amount as sales_amount
from payment as p
inner join rental as r on p.rental_id = r.rental_id
inner join inventory as i on r.inventory_id = i.inventory_id
"""
cursor.execute(sales_etl_query)
cnx.commit()


In [ ]:
cursor.execute("select * from factSales limit 5")
for record in cursor.fetchall():
  print(record)


# Analytics on Facts and Dimension Tables

# Top grossing movies

In [ ]:
top_gross_movie = """
select dm.title as movie, sum(fs.sales_amount) as revenue
from factSales as fs
inner join dimMovie as dm on fs.movie_key = dm.movie_key
group by movie
order by revenue desc
"""
cursor.execute(top_gross_movie)
result = cursor.fetchmany(10)
for record in result:
  print(record)


# Top grossing cities

In [ ]:
top_gross_city = """
select ds.city as city, sum(fs.sales_amount) as revenue
from factSales as fs
inner join dimStore as ds on fs.movie_key = ds.store_key
group by city
order by revenue desc
"""
cursor.execute(top_gross_city)
result = cursor.fetchmany(10)
for record in result:
  print(record)


# Revenue of movie by city, customer and month

In [ ]:
revenue_per_movie_city_customer_month = """
select
  dm.title as movie,
  dc.city as city,
  concat_ws(' ', dc.first_name, dc.last_name) as customer,
  dd.month_ as month,
  sum(fs.sales_amount) as revenue
from factSales as fs
inner join dimMovie as dm on fs.movie_key = dm.movie_key
inner join dimCustomer as dc on fs.customer_key = dc.customer_key
inner join dimDate as dd on fs.date_key = dd.date_key
group by movie, city, customer, month
order by revenue desc
"""
cursor.execute(revenue_per_movie_city_customer_month)
result = cursor.fetchmany(10)
for record in result:
  print(record)

# OLAP Work


## Roll Up
Calculates revenue sales by day rating and country. We are interested in country associated with the customer purchasing the movie.

In [ ]:
# Need to use sales, movie, date, and customer table
roll_up = """
select
	dd.day_ as day,
	dm.rating as rating,
	dc.country as country,
	sum(fs.sales_amount) as revenue
from factSales as fs
inner join dimDate as dd on fs.date_key = dd.date_key
inner join dimMovie as dm on dm.movie_key = fs.movie_key
inner join dimCustomer as dc on dc.customer_key = fs.customer_key
group by day, rating, country
order by revenue desc
limit 20;
"""
cursor.execute(roll_up)
result = cursor.fetchmany(10)
for record in result:
  print(record)

# Could also use query below by joining store table instead of customer. This will breakdown by country
# where customer bought rather store sold.
# select
#   dd.day_ as day,
#   dm.rating as rating,
#   ds.country as country,
#   sum(fs.sales_amount) as revenue
# from factSales as fs
# inner join dimDate as dd on fs.date_key = dd.date_key
# inner join dimMovie as dm on dm.movie_key = fs.movie_key
# inner join dimStore as ds on ds.store_key  = fs.store_key
# group by day, rating, country
# order by revenue desc

## Drill Down


In [ ]:
drill_down = """
select
	dd.day_ as day,
	dm.rating as rating,
	dc.district  as district,
	sum(fs.sales_amount) as revenue
from factSales as fs
inner join dimDate as dd on fs.date_key = dd.date_key
inner join dimMovie as dm on dm.movie_key = fs.movie_key
inner join dimCustomer as dc on dc.customer_key  = fs.customer_key
group by day, rating, district
order by revenue desc
"""
cursor.execute(drill_down)
result = cursor.fetchmany(10)
for record in result:
  print(record)


## Slice

Calculates revenue sales by day rating and country and slices the ratings dimension by looking at only PG-13 rated movies.

In [ ]:
slice_ratings = """
select
	dd.day_ as day,
	dm.rating as rating,
	dc.country as country,
	sum(fs.sales_amount) as revenue
from factSales as fs
inner join dimDate as dd on fs.date_key = dd.date_key
inner join dimMovie as dm on dm.movie_key = fs.movie_key
inner join dimCustomer as dc on dc.customer_key  = fs.customer_key
where dm.rating = "PG-13"
group by day, rating, country
order by revenue desc
limit 20;
"""
cursor.execute(slice_ratings)
result = cursor.fetchmany(10)
for record in result:
  print(record)

## Dice

In [ ]:
dice = """
select
	dd.day_ as day,
	dm.rating as rating,
	dc.country as country,
	sum(fs.sales_amount) as revenue
from factSales as fs
inner join dimDate as dd on fs.date_key = dd.date_key
inner join dimMovie as dm on dm.movie_key = fs.movie_key
inner join dimCustomer as dc on dc.customer_key  = fs.customer_key
where
	dm.rating in ("PG-13", "PG") and
	dc.city in ("Bellevue", "Lancaster") and
	dd.day_ in (1, 15, 30)
group by day, rating, country
order by revenue desc
limit 20;
"""
cursor.execute(dice)
result = cursor.fetchmany(10)
for record in result:
  print(record)

## Grouping Sets

Rollup month and country when performing group by.

In [ ]:
group_sets = """
select
	dd.month_ as month,
	dc.country as country,
	sum(fs.sales_amount) as revenue
from factSales as fs
inner join dimDate as dd on fs.date_key = dd.date_key
inner join dimCustomer as dc on fs.customer_key = dc.customer_key
group by month, country with rollup
order by revenue desc;
"""
cursor.execute(group_sets)
result = cursor.fetchmany(10)
for record in result:
  print(record)